# Relational Algebra Operators

In [1]:
import pandas as pd

def sigma(df: pd.DataFrame, func) -> pd.DataFrame:
    """
    Selection operator : σ, to filter rows
    Arguments:
        df: the dataframe to filter
        func: a lambda function that return a true or false boolean if the row is to be kept or not
        example : sigma(t, lambda df: df["gender"] == "female" and (df["pizza"] == "mushroom" or df["pizza"] == "pepperoni"))
    """
    
    return pd.DataFrame(df[df.apply(func, axis=1)])


def pi(df: pd.DataFrame, columns: list) -> pd.DataFrame:    
    """
    Projection operator : ∏, to select columns
    Arguments:
        df: the dataframe to filter
        columns: a list of column names (ex. [RACE, AGE])
    """        
    return pd.DataFrame(df[columns]).drop_duplicates().reset_index(drop=True)

def rho(df: pd.DataFrame, current_col_name: str, new_col_name: str) -> pd.DataFrame:    
    """
    Renaming operator : ρ, to change a column name
    Arguments:
        df: the dataframe to modify
        current_col_name: the name to change
        new_col_name: the new name
    """ 
    
    df = df.rename(columns={current_col_name: new_col_name})
    
    return df

def union(dfA: pd.DataFrame, dfB: pd.DataFrame) -> pd.DataFrame:
    """
    Union operator : ∪, to get A and B elements without duplicates
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
    """        
    return pd.concat([dfA,dfB]).drop_duplicates().reset_index(drop=True)

def intersection(dfA: pd.DataFrame, dfB: pd.DataFrame) -> pd.DataFrame:
    """
    Intersection operator : ∩, to get elements which are in A and B
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
    """        
    return pd.merge(dfA, dfB, how='inner')

def substraction(dfA: pd.DataFrame, dfB: pd.DataFrame) -> pd.DataFrame:
    """
    Substraction operator : -, to get elements which are in A and not in B
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
    """  
    df_all = dfA.merge(dfB, how='left', indicator=True)
    df_all = df_all[df_all["_merge"]=="left_only"].iloc[:,:-1]
    return pd.DataFrame(df_all)

def cartesian_product(dfA: pd.DataFrame, dfB: pd.DataFrame) -> pd.DataFrame:
    """
    Cartesian product operator : X, if it's of any use
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
    """  
    return dfA.merge(dfB, how='cross')

def division(dfA, dfB, keyA, keyB):
    
    # Create three sets, with unique values of selected columns in A and in B
    setA = set(dfA[keyA])
    setB = set(dfB[keyB])
    
    # Create an empty set to store results
    setC = set()
    
    # For each value in set one
    for A in setA:
        
        # Get only rows with value A
        temp = dfA[dfA[keyA]==A]
        
        # If unique values of Temp is a superset of setB
        if set(temp[keyB]) >= setB:
            
            # Add A to result
            setC.add(A)
            
    return pd.DataFrame({keyA: list(setC)})

def join(dfA: pd.DataFrame, dfB: pd.DataFrame, key: str) -> pd.DataFrame:
    """
    Join : ⋈, to add columns from another dataframe using a key
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
        key: the common column name to use to merge
    """  
    return pd.merge(dfA, dfB, on=key)

def theta_join(dfA, dfB, func) -> pd.DataFrame:
    """
    Theta Join : ⋈θ, to filter a cartesian product on a condition
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
        func: a function which can use columns of dfA and dfB to return a boolean
    """ 
    
    temp = cartesian_product(dfA, dfB)
    
    return pd.DataFrame(temp[temp.apply(f, axis=1)])

def semi_join(dfA: pd.DataFrame, dfB: pd.DataFrame, key: str) -> pd.DataFrame:
    
    """
    Semi-join : ⋉, to keep rows in which one column's value is in another table's column
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
        key: the column on which to filter
    """ 
    setB = set(dfB[key])
    return pd.DataFrame(dfA[dfA[key].isin(setB)])

def anti_join(dfA: pd.DataFrame, dfB: pd.DataFrame, key: str) -> pd.DataFrame:
    """
    Anti-join : ▷, to exclude rows in which one column's value is in another table's column
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
        key: the column on which to filter
    """ 
    setB = set(dfB[key])
    return pd.DataFrame(dfA[dfA[key].isin(setB) == False])

def left_outer_join(dfA: pd.DataFrame, dfB: pd.DataFrame, key: str) -> pd.DataFrame:
    """
    Left Outer Join : ⟕, to allow empty values of B
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
        key: the joining key
    """ 
    return pd.merge(dfA, dfB, on=key, how='left')

def right_outer_join(dfA: pd.DataFrame, dfB: pd.DataFrame, key: str) -> pd.DataFrame:
    """
    Right Outer Join : ⟖, to allow empty values of A
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
        key: the joining key
    """ 
    return pd.merge(dfA, dfB, on=key, how='right')

def full_outer_join(dfA: pd.DataFrame, dfB: pd.DataFrame, key: str) -> pd.DataFrame:
    """
    Full Outer Join : ⟗, to allow empty values of A and B
    Arguments:
        dfA: the first dataframe
        dfB: the second dataframe
        key: the joining key
    """ 
    return pd.merge(dfA, dfB, on=key, how='outer')

# Exercices

source : http://openclassroom.stanford.edu/MainFolder/courses/cs145/old-site/docs/backup/ra-exercises.html

In [2]:
pizza_json = """{ "Person":[ { "name": "Amy", "age": 16, "gender": "female" }, { "name": "Ben", "age": 21, "gender": "male" }, { "name": "Cal", "age": 33, "gender": "male" }, { "name": "Dan", "age": 13, "gender": "male" }, { "name": "Eli", "age": 45, "gender": "male" }, { "name": "Fay", "age": 21, "gender": "female" }, { "name": "Gus", "age": 24, "gender": "male" }, { "name": "Hil", "age": 30, "gender": "female" }, { "name": "Ian", "age": 18, "gender": "male" } ], "Frequents":[ { "name": "Amy", "pizzeria": "Pizza Hut" }, { "name": "Ben", "pizzeria": "Pizza Hut" }, { "name": "Ben", "pizzeria": "Chicago Pizza" }, { "name": "Cal", "pizzeria": "Straw Hat" }, { "name": "Cal", "pizzeria": "New York Pizza" }, { "name": "Dan", "pizzeria": "Straw Hat" }, { "name": "Dan", "pizzeria": "New York Pizza" }, { "name": "Eli", "pizzeria": "Straw Hat" }, { "name": "Eli", "pizzeria": "Chicago Pizza" }, { "name": "Fay", "pizzeria": "Dominos" }, { "name": "Fay", "pizzeria": "Little Caesars" }, { "name": "Gus", "pizzeria": "Chicago Pizza" }, { "name": "Gus", "pizzeria": "Pizza Hut" }, { "name": "Hil", "pizzeria": "Dominos" }, { "name": "Hil", "pizzeria": "Straw Hat" }, { "name": "Hil", "pizzeria": "Pizza Hut" }, { "name": "Ian", "pizzeria": "New York Pizza" }, { "name": "Ian", "pizzeria": "Straw Hat" }, { "name": "Ian", "pizzeria": "Dominos" } ], "Eats":[ { "name": "Amy", "pizza": "pepperoni" }, { "name": "Amy", "pizza": "mushroom" }, { "name": "Ben", "pizza": "pepperoni" }, { "name": "Ben", "pizza": "cheese" }, { "name": "Cal", "pizza": "supreme" }, { "name": "Dan", "pizza": "pepperoni" }, { "name": "Dan", "pizza": "cheese" }, { "name": "Dan", "pizza": "sausage" }, { "name": "Dan", "pizza": "supreme" }, { "name": "Dan", "pizza": "mushroom" }, { "name": "Eli", "pizza": "supreme" }, { "name": "Eli", "pizza": "cheese" }, { "name": "Fay", "pizza": "mushroom" }, { "name": "Gus", "pizza": "mushroom" }, { "name": "Gus", "pizza": "supreme" }, { "name": "Gus", "pizza": "cheese" }, { "name": "Hil", "pizza": "supreme" }, { "name": "Hil", "pizza": "cheese" }, { "name": "Ian", "pizza": "supreme" }, { "name": "Ian", "pizza": "pepperoni" } ], "Serves":[ { "pizzeria": "Pizza Hut", "pizza": "pepperoni", "price": 12 }, { "pizzeria": "Pizza Hut", "pizza": "sausage", "price": 12 }, { "pizzeria": "Pizza Hut", "pizza": "cheese", "price": 9 }, { "pizzeria": "Pizza Hut", "pizza": "supreme", "price": 12 }, { "pizzeria": "Little Caesars", "pizza": "pepperoni", "price": 9.75 }, { "pizzeria": "Little Caesars", "pizza": "sausage", "price": 9.5 }, { "pizzeria": "Little Caesars", "pizza": "cheese", "price": 7 }, { "pizzeria": "Little Caesars", "pizza": "mushroom", "price": 9.25 }, { "pizzeria": "Dominos", "pizza": "cheese", "price": 9.75 }, { "pizzeria": "Dominos", "pizza": "mushroom", "price": 11 }, { "pizzeria": "Straw Hat", "pizza": "pepperoni", "price": 8 }, { "pizzeria": "Straw Hat", "pizza": "cheese", "price": 9.25 }, { "pizzeria": "Straw Hat", "pizza": "sausage", "price": 9.75 }, { "pizzeria": "New York Pizza", "pizza": "pepperoni", "price": 8 }, { "pizzeria": "New York Pizza", "pizza": "cheese", "price": 7 }, { "pizzeria": "New York Pizza", "pizza": "supreme", "price": 8.5 }, { "pizzeria": "Chicago Pizza", "pizza": "cheese", "price": 7.75 }, { "pizzeria": "Chicago Pizza", "pizza": "supreme", "price": 8.5 } ] }"""

In [3]:
import json

_dict = json.loads(pizza_json)

# Kind of pizza people like to eat : name, pizza
Eats = pd.json_normalize(_dict['Eats']) 

# People caracteristics : name, age, gender
Person = pd.json_normalize(_dict['Person'])

# Where people go to eat : name, pizzeria
Frequents = pd.json_normalize(_dict['Frequents']) 

# Prices of pizza in each restaurant : pizzeria, pizza, price
Serves = pd.json_normalize(_dict['Serves']) 

a. Find all pizzerias frequented by at least one person under the age of 18.

b. Find the names of all females who eat either mushroom or pepperoni pizza (or both).

c. Find the names of all females who eat both mushroom and pepperoni pizza.

d. Find all pizzerias that serve at least one pizza that Amy eats for less than $10.00.

e. Find all pizzerias that are frequented by only females or only males.

f. For each person, find all pizzas the person eats that are not served by any pizzeria the person frequents. Return all such person (name) / pizza pairs.

g. Find the names of all people who frequent only pizzerias serving at least one pizza they eat.

h. Find the names of all people who frequent every pizzeria serving at least one pizza they eat.

In [4]:
# a. Find all pizzerias frequented by at least one person under the age of 18.
sigma(join(Frequents, Person, "name"), lambda df: df["age"] < 18)

,name,pizzeria,age,gender
0,Amy,Pizza Hut,16,female
5,Dan,Straw Hat,13,male
6,Dan,New York Pizza,13,male


In [5]:
# b. Find the names of all females who eat either mushroom or pepperoni pizza (or both).
t = join(Person, Eats, "name")

sigma(t, lambda df: df["gender"] == "female" and (df["pizza"] == "mushroom" or df["pizza"] == "pepperoni"))

,name,age,gender,pizza
0,Amy,16,female,pepperoni
1,Amy,16,female,mushroom
12,Fay,21,female,mushroom


In [6]:
# c. Find the names of all females who eat both mushroom and pepperoni pizza.
a = join(Person, Eats, "name")
b = sigma(a, lambda df: df["gender"] == "female" and df["pizza"]=="mushroom")
c = sigma(a, lambda df: df["gender"] == "female" and df["pizza"]=="pepperoni")

union(b, c)

,name,age,gender,pizza
0,Amy,16,female,mushroom
1,Fay,21,female,mushroom
2,Amy,16,female,pepperoni


In [7]:
# d. Find all pizzerias that serve at least one pizza that Amy eats for less than $10.00.
a = join(Eats, Serves, "pizza")
a = sigma(a, lambda df: df["name"] == "Amy" and df["price"] < 10)

pi(a, "pizzeria")

,pizzeria
0,Little Caesars
1,Straw Hat
2,New York Pizza


In [291]:
# e. Find all pizzerias that are frequented by only females or only males.
a = join(Frequents, Person, "name")
b = division(a, pi(a, "gender"), "pizzeria", "gender")

substraction(pi(Frequents, "pizzeria"), b)

,pizzeria
1,Chicago Pizza
3,New York Pizza
5,Little Caesars


In [292]:
# f. For each person, find all pizzas the person eats that are not served by any pizzeria the person frequents.
# Return all such person (name) / pizza pairs.

# Pizza people might want to eat
A = pi(join(Person, Eats, "name"), ["name", "pizza"])

# Pizza people have access to
B = pi(join(Frequents, Serves, "pizzeria"), ["name", "pizza"])

substraction(A, B)

,name,pizza
1,Amy,mushroom
9,Dan,mushroom
13,Gus,mushroom


In [293]:
# g. Find the names of all people who frequent only pizzerias serving at least one pizza they eat.

# Pizzerias people frequent
a = Frequents

# Pizzerias where people can eat a pizza
b = pi(join(Eats, Serves, "pizza"), ["name", "pizzeria"])

# People who frequents pizzeria they can't eat pizza in
c = pi(substraction(a, b), "name")

# Other people
substraction(pi(Person, "name"), c)

,name
0,Amy
1,Ben
3,Dan
4,Eli
5,Fay
6,Gus
7,Hil


In [294]:
# h. Find the names of all people who frequent every pizzeria serving at least one pizza they eat.

# Pizzerias people frequent
a = Frequents

# Pizzerias people can eat pizza in
b = pi(join(Eats, Serves, "pizza"), ["name", "pizzeria"])

c = substraction(b, a)
substraction(pi(Person, "name"), pi(c, "name"))

,name
5,Fay


In [9]:
Eats

,name,pizza
0,Amy,pepperoni
1,Amy,mushroom
2,Ben,pepperoni
3,Ben,cheese
4,Cal,supreme
5,Dan,pepperoni
6,Dan,cheese
7,Dan,sausage
8,Dan,supreme
9,Dan,mushroom
